In [ ]:
import base64
import requests
import os

def encode_image_to_base64(image_path):
    """
    读取图片文件并将其编码为base64字符串。

    :param image_path: 图片文件的本地路径
    :return: base64编码的图片字符串
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def analyze_image_gpt4o(prompt, image_path):
    """
    使用GPT-4o模型分析图片并返回描述。

    :param prompt: 提示词
    :param image_path: 图片文件的本地路径
    :return: 模型生成的描述或错误信息
    """
    # OpenAI API endpoint
    url = "https://api.openai.com/v1/chat/completions"
    
    # 编码图片
    encoded_image = encode_image_to_base64(image_path)
    
    # 准备payload
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{encoded_image}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 300
    }
    
    # 设置headers
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"
    }
    
    # 发送API请求
    response = requests.post(url, json=payload, headers=headers)
    
    # 检查请求是否成功
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Error: {response.status_code}, {response.text}"


result = analyze_image_gpt4o("Describe this image in detail.", "Project/pdf2lec_llm/data/test_images/屏幕截图 2024-10-19 164116.png")
print(result)